# TF slicing

* [Introduction to tensor slicing](https://www.tensorflow.org/guide/tensor_slicing)

> In this guide, you will learn how to use the TensorFlow APIs to:
> * Extract slices from a tensor
> * Insert data at specific indices in a tensor

In [3]:
import numpy as np
import tensorflow as tf

# References


### [tf.slice](https://www.tensorflow.org/api_docs/python/tf/slice)

```
tf.slice(
    input_, 
    begin,            # Coordinate (index subscripts) of the start point
    size,             # Shape of the region to slice/dice (NOT size) 
    name=None
)
```

> Extracts a slice from a tensor.

### [tf.Tensor.__get_item__](https://www.tensorflow.org/api_docs/python/tf/Tensor#__getitem__)

```
__getitem__(
    slice_spec, var=None
)
```
> Insert another dimension
```
foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
print(foo[tf.newaxis, :, :].eval()) # => [[[1,2,3], [4,5,6], [7,8,9]]]
print(foo[:, tf.newaxis, :].eval()) # => [[[1,2,3]], [[4,5,6]], [[7,8,9]]]
print(foo[:, :, tf.newaxis].eval()) # => [[[1],[2],[3]], [[4],[5],[6]],
[[7],[8],[9]]]
```
> Ellipses (3 equivalent operations)
```
foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
print(foo[tf.newaxis, :, :].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
print(foo[tf.newaxis, ...].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
print(foo[tf.newaxis].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
```


### [tf.strided_slice](https://www.tensorflow.org/api_docs/python/tf/strided_slice)

```
tf.strided_slice(
    input_, begin, end, strides=None, begin_mask=0, end_mask=0, ellipsis_mask=0,
    new_axis_mask=0, shrink_axis_mask=0, var=None, name=None
)
```
> Extract slices of tensors **by 'striding' over** the tensor dimensions.



# Constraint

## No NumPy advance-indexing equivalent

NumPy indexing allows the combination of slice and array. TF slicing only allow either:
1. NumPy/Pythonic slice syntax
2. Scalar element selection with scatter indices

# Pythonic/NumPy-like slice syntax

## 1D slice

<img src="image/tf_slicing_1d_1.png" align="left" width=300 />


In [15]:
t1 = tf.constant([0, 1, 2, 3, 4, 5, 6, 7])
tf.slice(    # Same with t1[1:4]
    t1,
    begin=[1],
    size=[3]
)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

<img src="image/tf_slicing_1d_3.png" align="left" width=300 />

In [6]:
t1[::3]

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 3, 6], dtype=int32)>

## Regional (2D, 3D, ...) slice

## 2D
<img src="image/tf_slicing_2d.png" align="left" width=200 />

In [22]:
t2 = tf.reshape(tf.range(20,dtype=tf.int32), shape=(4,5))
tf.slice(
    t2,
    begin=[0,1],   # Coordinate (n,d) as the start point
    size=[3,2]     # Shape (3,2) -> (n+3, n+2) as the end point
)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 1,  2],
       [ 6,  7],
       [11, 12]], dtype=int32)>

In [24]:
# Same with
t2[
    :-1, # [0:-1)
    1:3  # [0:3)
]

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 1,  2],
       [ 6,  7],
       [11, 12]], dtype=int32)>

## 3D

In [26]:
t = tf.constant([
    [
        [1, 1, 1], 
        [2, 2, 2]
    ],
    [
        [3, 3, 3], 
        [4, 4, 4]
    ],
    [
        [5, 5, 5], 
        [6, 6, 6]
    ]
])

tf.slice(
    t, 
    [1, 0, 0],                     # Start point coordinate 
    [1, 1, 3]                      # Shape of the cube to dice
)                                  # [[[3, 3, 3]]]

<tf.Tensor: shape=(1, 1, 3), dtype=int32, numpy=array([[[3, 3, 3]]], dtype=int32)>

In [28]:
tf.slice(
    t, 
    [1, 0, 0], 
    [1, 2, 3]
)                                  # [[[3, 3, 3],

<tf.Tensor: shape=(1, 2, 3), dtype=int32, numpy=
array([[[3, 3, 3],
        [4, 4, 4]]], dtype=int32)>

In [27]:
tf.slice(
    t, 
    [1, 0, 0], 
    [2, 1, 3]
)                                  # [[[3, 3, 3]],
                                   #  [[5, 5, 5]]]

<tf.Tensor: shape=(2, 1, 3), dtype=int32, numpy=
array([[[3, 3, 3]],

       [[5, 5, 5]]], dtype=int32)>

---
# Element-wise indices (axes junctions)

List of indices where **each index identifies a scalar element** in a Tensor.

> To extract slices from multiple axes of a tensor, use tf.gather_nd. This is useful when you want to gather the elements of a matrix as opposed to just its rows or columns.

In [30]:
x = tf.constant(np.arange(24).reshape(4,6))
x

<tf.Tensor: shape=(4, 6), dtype=int64, numpy=
array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]])>

In [31]:
tf.gather_nd(x, indices=[
    [0,1],        # Each index is a scalar element coordinate (row, col) = (0,1) 
    [1,3],
    [2,5],
    [3,0]
])

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 1,  9, 17, 18])>